In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 
import spotipy 
import librosa
import librosa.display
import numpy as np
import matplotlib as mpl
import urllib.request
import seaborn as sns
import tqdm
import scipy
import joblib
import os.path
from pathlib import Path
from IPython.display import Audio, Markdown, Image
from spotipy.oauth2 import SpotifyClientCredentials

# Números y Datos
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.interpolate import interp1d

# Análisis de sonido
import librosa
import librosa.display
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 

# Machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from sklearn.preprocessing import quantile_transform

# Styles
sns.set_context('poster')
sns.set_style('darkgrid')

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [ ]:
# Extraemos los dataframes de nuestros files
df_tracks = pd.read_pickle('../sources/tracks.pickle')
df_af = pd.read_pickle('../sources/audio_features.pickle')
df_aa = pd.read_pickle('../sources/audio_analysis.pickle') \
    .set_index(['id','start']) \
    .sort_values(by=['id', 'start'], ascending=True, na_position='first')

In [ ]:
# Generamos un dataframe combinando la data de tracks y de audio features
df_merged = df_tracks.merge(
    df_af, 
    on='id', 
    how='left'
)
df_merged = df_merged.drop(
    [
     'type_x',
     'type_y',
     'uri_x',
     'uri_y',
     'track_href',
     'analysis_url',
     'href',
     'preview_url',
     'external_ids',
     'duration_ms_y'
    ], 
    1
)

df_merged.rename(columns = {"duration_ms_x": "duration_ms"}, inplace = True)

In [ ]:
# Extraemos para cada registro la variable release_date y la asignamos a una nueva columna en nuestro dataframe.

# Función que generamos para formatear la fecha
def date_formator(date):
    if '-' in date:
        year = pd.to_datetime(date, format = '%Y-%m-%d').year
        return int(year)
    elif int(date)>0:
        year = pd.to_datetime(date, format = '%Y').year
        return int(year)
    return None


df_merged['release_date'] = df_merged['album'].map(lambda x: x['release_date'])
df_merged['release_date'] = df_merged['release_date'].apply(date_formator)
df_merged['release_date'].head()

In [ ]:
# Obtengo ids de album
df_merged['album_id'] = df_merged['album'].map(lambda x: x['id'])

# Agrego artistas y género
df_merged[["artists","genre"]] =  df_merged.loc[:,["artists","genre"]] 

In [ ]:
# Vemos cuales son nuestras columnas
df_merged.columns.values

In [ ]:
df_merged.sample(3).T

In [ ]:
def plot_radar_chart_stats(s25,s50,s75,color,ax):
    grupo=s25.name
    labels = s25.index

    angles=np.linspace(0, 2*np.pi, len(s25), endpoint=False)
    angles=np.append(angles,angles[0])
    s25=np.append(s25,s25[0])
    s50=np.append(s50,s50[0])
    s75=np.append(s75,s75[0])

    ax.plot(angles, s50, 'o-', linewidth=1,color=color,markersize=4)
    ax.set_yticklabels([])
    ax.set_thetagrids(angles * 180/np.pi, labels, fontsize=6, color='gray')
    ax.tick_params(pad=-6, direction='out', length=8, color='k', zorder=-1)
    ax.fill_between(angles, s25, s75, alpha=0.2,color=color)
    ax.set_title(grupo, fontsize=12, loc='left')
    ax.grid(True)
    ax.set_ylim(0,1)


In [ ]:
# quantil=50 # 50 quantiles
# df_num = df_merged.copy()
# df_num = df_num.select_dtypes(['number']).drop(['disc_number','track_number','release_date','duration_ms','mode','time_signature'],1)

# quantile_transform(
#         df_num, 
#         n_quantiles = 100,
#         output_distribution = 'uniform', 
#         random_state = 0, 
#         copy = False
#         )

In [ ]:
# df_merged.select_dtypes(['number']).drop(['disc_number','track_number','release_date','duration_ms'],1).head()
# df_merged['mode'][1:200]

In [ ]:
quantil=100 # 50 quantiles
df_num = df_merged.copy()
df_num = df_num.select_dtypes(['number']).drop(['disc_number','track_number','release_date','duration_ms','mode','time_signature'],1)

df_num = pd.DataFrame(
    quantile_transform(
        df_num, 
        n_quantiles = quantil,
        output_distribution = 'uniform', 
        random_state = 0, 
        copy = True
    ), columns=df_num.columns,index=df_num.index
)
df_num['genre']=df_merged['genre']

normalized_stats_25 = df_num.groupby('genre').quantile(.25)
normalized_stats_50 = df_num.groupby('genre').quantile(.5)
normalized_stats_75 = df_num.groupby('genre').quantile(.75)
df_num.head(20)

In [ ]:
rows=2
my_palette = sns.color_palette("Set2")
q = int(len(normalized_stats_25)/rows)+len(normalized_stats_25)%rows
fig, axs = plt.subplots(rows, q, subplot_kw=dict(polar=True))

for i in range(rows*q):
    if i<len(normalized_stats_25):
        plot_radar_chart_stats(normalized_stats_25.iloc[i],
                               normalized_stats_50.iloc[i],
                               normalized_stats_75.iloc[i],
                               my_palette[i%8],axs[i%rows, int(i/rows)])
    else:
        fig.delaxes(axs[i%rows, int(i/rows)])

plt.savefig('radar_chart.png',dpi=800, bbox_inches = 'tight', pad_inches = 0)
plt.show()

Gráfico Radar de los features por cada género del set de datos.

La línea gruesa indica la mediana, y el sombreado es el rango intercuartil.

Las características se encuentran escaladas y normalizadas por cuantiles (dando una medición robusta de cada feature para hacer las estadísticas comparables)

Análisis de este gráfico
* La ópera y la clásica son los que más acousticness tienen.
* Las canciones de trance son las que en general suelen tener mayor duración (seguidas por las ambient y drum-and-bass).
* La música ambient y classical son las de mayor instrumentalness.
* Death-metal suelen ser el género de mayor liveness junto con drum-and-bass.
* Death-metal es incuestionablemente el género más energético, seguido por drum-and-bass, ska y trance.
* Opera y classical comparten las característica de baja energy, baja valence, bajo tempo, alto acousticness y bajo speechness.
* Los géneros más danceable son singer-songwriter, trance, jazz y ska.
* La variable de popularidad corre a lo largo de todo el espectro para la mayoría de los géneros y ninguno muestra una amplia diferencia con respecto al resto. Más allá de eso, las canciones de singer-songwriter tienen (en media) las canciones más populares, seguidos por jazz y classical.

Desdoblamos los features de pitches y timbre en distintas columnas

In [ ]:
df_m2 = df_aa

pitches_nombre = ['p00_C','p01_C#','p02_D','p03_D#','p04_E','p05_F',
                 'p06_F#','p07_G','p08_G#','p09_A','p10_A#','p11_B']
df1 = pd.DataFrame(df_m2['pitches'].tolist(), columns=pitches_nombre, index=df_m2.index)
df_m2 = pd.concat([df_m2,df1],axis=1)

timbre_nombre = ['t00','t01','t02','t03','t04','t05',
                 't06','t07','t08','t09','t10','t11']
df1 = pd.DataFrame(df_m2['timbre'].tolist(), columns=timbre_nombre, index=df_m2.index)
df_m2 = pd.concat([df_m2,df1],axis=1).drop(['pitches','timbre'],1)

df1 = None
df_m2.head(3).T

Extraemos los datos de MEDIANA + IQR (RANGO INTERCUARTIL)

In [ ]:
grouped = df_m2.drop(['duration'],1).groupby(level='id')
medianas = grouped.quantile(.5)
iqr = grouped.quantile(.75)-grouped.quantile(.25)
medias = grouped.mean()
desvios = grouped.std()

In [ ]:
df_pitches_timbres_medias_std = pd.merge(medias, desvios, on='id', suffixes=('_media', '_std'))
df_pitches_timbres_medianas_iqr = pd.merge(medianas, iqr, on='id', suffixes=('_mediana', '_iqr'))

Y LO NORMALIZAMOS (al menos hasta donde puede hacerse de una manera sencilla)

In [ ]:
quantil=100 # 50 quantiles

features_2 = df_pitches_timbres_medianas_iqr.columns
df_pitches_timbres_medianas_iqr_normalizado_quantiles = pd.DataFrame(
    quantile_transform(
        df_pitches_timbres_medianas_iqr[features_2], 
        n_quantiles = quantil,
        output_distribution = 'normal', 
        random_state = 0, 
        copy = True
    ), columns=features_2,index=medianas.index
)

df_pitches_timbres_medianas_iqr_normalizado_quantiles.head(3).T

Mergeamos los datos normalizados con el dataframe

In [ ]:
df_merged2 = df_merged.select_dtypes(['number']).drop(['disc_number','track_number','release_date'],1)

df_merged_quantiles = pd.DataFrame(
    quantile_transform(
        df_merged2,
        n_quantiles = quantil,
        output_distribution = 'normal', 
        random_state = 0, 
        copy = True
    ), columns=df_merged2.columns,index=df_merged2.index
)
df_merged_quantiles['genre']=df_merged['genre']

df_merged_quantiles.head(3).T

Lo integramos al resto de los datos y lo guardamos.  Fin.

In [ ]:
df_merged_all = df_merged_quantiles.merge(
    df_pitches_timbres_medianas_iqr_normalizado_quantiles, 
    on='id', 
    how='inner'
)
df_merged_all.head(3).T

In [ ]:
df_merged_all.to_pickle("../sources/df_merged_all_quantiles_mediana_iqr.pickle")

In [ ]:
df_merged_all.head(3).T